In [1]:
import pandas as pd
from datetime import datetime
import os
pd.set_option('display.max_rows',90)
pd.set_option('display.max_columns',90)

In [2]:
def mostra_hora():
    agora = datetime.now()
    print(agora.strftime('%Y-%m-%d %H:%M:%S'))

In [3]:
mostra_hora()
from os.path import expanduser, join, abspath

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# warehouse_location points to the default location for managed databases and tables
warehouse_location = abspath('spark-warehouse')

config = SparkConf().setAll([('spark.executor.memory', '8g'), 
                             ('spark.executor.cores', '6'), 
                             ('spark.cores.max', '6'), 
                             ('spark.driver.memory','8g'),
                             ('spark.driver.allowMultipleContexts','true')])

spark = SparkSession \
   .builder \
   .appName("fact_client") \
   .config("spark.sql.warehouse.dir", warehouse_location) \
   .config("hive.exec.dynamic.partition", "true") \
   .config("hive.exec.dynamic.partition.mode", "nonstrict")\
   .master("yarn") \
   .config(conf=config)  \
   .enableHiveSupport() \
   .getOrCreate()
mostra_hora()

2020-08-13 09:13:16
2020-08-13 09:13:38


In [4]:
spark.sql('''set hive.exec.dynamic.partition.mode=nonstrict''')
mostra_hora()

2020-08-13 09:13:39


In [5]:
# Definicao de variaveis
mostra_hora()
var_safra=202008

var_safraM01 = spark.sql('''select date_format(add_months(to_date(from_unixtime(unix_timestamp(concat({var_safra},01),"yyyyMMdd"))),-1 ),"yyyyMM") as safra_m01'''.format(var_safra=var_safra)).first()[0]
print('# safra_M01 => ' + str(var_safraM01))

var_safraM03 = spark.sql('''select date_format(add_months(to_date(from_unixtime(unix_timestamp(concat({var_safra},01),"yyyyMMdd"))),-3 ),"yyyyMM") as safra_m03'''.format(var_safra=var_safra)).first()[0]
print('# safra_M03 => ' + str(var_safraM03))

var_safraM06 = spark.sql('''select date_format(add_months(to_date(from_unixtime(unix_timestamp(concat({var_safra},01),"yyyyMMdd"))),-6 ),"yyyyMM") as safra_m06'''.format(var_safra=var_safra)).first()[0]
print('# safra_M06 => ' + str(var_safraM06))

var_safraM12 = spark.sql('''select date_format(add_months(to_date(from_unixtime(unix_timestamp(concat({var_safra},01),"yyyyMMdd"))),-12 ),"yyyyMM") as safra_m12'''.format(var_safra=var_safra)).first()[0]
print('# safra_M12 => ' + str(var_safraM12))

mostra_hora()

2020-08-13 09:13:39
# safra_M01 => 202007
# safra_M03 => 202005
# safra_M06 => 202002
# safra_M12 => 201908
2020-08-13 09:13:46


In [6]:
# Trata TBL_FACT_CLIENT
df_fact_client=spark.sql('''
select num_document
      ,{var_safra} as safra
      ,max(vl_ticket_acum_m0)  as vl_ticket_acum_m0
      ,max(qty_ticket_acum_m0) as qty_ticket_acum_m0
      ,max(vl_ticket_acum_m1)  as vl_ticket_acum_m1
      ,max(qty_ticket_acum_m1) as qty_ticket_acum_m1
      ,max(vl_ticket_acum_m3)  as vl_ticket_acum_m3
      ,max(qty_ticket_acum_m3) as qty_ticket_acum_m3
      ,max(vl_ticket_acum_m6)  as vl_ticket_acum_m6
      ,max(qty_ticket_acum_m6) as qty_ticket_acum_m6
      ,max(vl_ticket_acum_m12)  as vl_ticket_acum_m12
      ,max(qty_ticket_acum_m12) as qty_ticket_acum_m12
      from (
select
   num_document
  ,sum(vl_ticket_m0) as vl_ticket_acum_m0
  ,sum(qty_ticket_m0) as qty_ticket_acum_m0
  ,NULL  as vl_ticket_acum_m1
  ,NULL  as qty_ticket_acum_m1
  ,NULL  as vl_ticket_acum_m3
  ,NULL  as qty_ticket_acum_m3
  ,NULL  as vl_ticket_acum_m6
  ,NULL  as qty_ticket_acum_m6
  ,NULL  as vl_ticket_acum_m12
  ,NULL  as qty_ticket_acum_m12
FROM db_analytics_crm.tbl_fact_client
where safra = {var_safra}
group by num_document
union all
select
   num_document
  ,NULL  as vl_ticket_acum_m0
  ,NULL  as qty_ticket_acum_m0
  ,sum(vl_ticket_m0) as vl_ticket_acum_m1
  ,sum(qty_ticket_m0) as qty_ticket_acum_m1
  ,NULL  as vl_ticket_acum_m3
  ,NULL  as qty_ticket_acum_m3
  ,NULL  as vl_ticket_acum_m6
  ,NULL  as qty_ticket_acum_m6
  ,NULL  as vl_ticket_acum_m12
  ,NULL  as qty_ticket_acum_m12
FROM db_analytics_crm.tbl_fact_client
where safra = {var_safraM01}
group by num_document
union all
select
   num_document
  ,NULL as vl_ticket_acum_m0
  ,NULL as qty_ticket_acum_m0
  ,NULL as vl_ticket_acum_m1
  ,NULL as qty_ticket_acum_m1
  ,sum(vl_ticket_m0)  as vl_ticket_acum_m3
  ,sum(qty_ticket_m0)  as qty_ticket_acum_m3
  ,NULL  as vl_ticket_acum_m6
  ,NULL  as qty_ticket_acum_m6
  ,NULL  as vl_ticket_acum_m12
  ,NULL  as qty_ticket_acum_m12
FROM db_analytics_crm.tbl_fact_client
where safra between {var_safraM03} and {var_safraM01}
group by num_document
union all
select
   num_document
  ,NULL as vl_ticket_acum_m0
  ,NULL as qty_ticket_acum_m0
  ,NULL as vl_ticket_acum_m1
  ,NULL as qty_ticket_acum_m1
  ,NULL  as vl_ticket_acum_m3
  ,NULL  as qty_ticket_acum_m3
  ,sum(vl_ticket_m0)  as vl_ticket_acum_m6
  ,sum(qty_ticket_m0)  as qty_ticket_acum_m6
  ,NULL  as vl_ticket_acum_m12
  ,NULL  as qty_ticket_acum_m12
FROM db_analytics_crm.tbl_fact_client
where safra between {var_safraM06} and {var_safraM01}
group by num_document
union all
select
   num_document
  ,NULL as vl_ticket_acum_m0
  ,NULL as qty_ticket_acum_m0
  ,NULL as vl_ticket_acum_m1
  ,NULL as qty_ticket_acum_m1
  ,NULL  as vl_ticket_acum_m3
  ,NULL  as qty_ticket_acum_m3
  ,NULL  as vl_ticket_acum_m6
  ,NULL  as qty_ticket_acum_m6
  ,sum(vl_ticket_m0)  as vl_ticket_acum_m12
  ,sum(qty_ticket_m0)  as qty_ticket_acum_m12
FROM db_analytics_crm.tbl_fact_client
where safra between {var_safraM12} and {var_safraM01}
group by num_document
) 
group by num_document
'''.format(var_safra=var_safra
          ,var_safraM01=var_safraM01
          ,var_safraM03=var_safraM03
          ,var_safraM06=var_safraM06
          ,var_safraM12=var_safraM12))
df_fact_client.createOrReplaceTempView('tb_df_fact_client_acum')
print('#')
print('# Dados da FACT_CLIENT processada')
print('#')
mostra_hora()

2020-08-13 09:13:50


spark.sql('''
desc tb_fact_client_acum_temp
''').show()
mostra_hora()

mostra_hora()
spark.sql('''
drop table if exists db_analytics_crm.tbl_fact_client_acum_temp4
''')
mostra_hora()

mostra_hora()
spark.sql('''
select safra, count(*)
from tb_fact_client_acum_temp
group by safra
order by safra desc
''').show(100,truncate=False)
mostra_hora()

----
# Tabela dim_product

In [8]:
mostra_hora()
print('#')
print('# Processando safra ' + str(var_safra))

df_join_product=spark.sql('''select
   num_document
  ,safra as safra
  ,nvl(max(tot_categ_food),0)     as tot_categ_food
  ,nvl(max(tot_categ_nfood),0)    as tot_categ_nfood
  ,nvl(max(vl_tot_categ_food),0)  as vl_tot_categ_food
  ,nvl(max(vl_tot_categ_nfood),0) as vl_tot_categ_nfood
from(
select
          it.num_document, it.safra
         ,COUNT(DISTINCT CASE WHEN UPPER(NVL(pd.cod_bus,"FOOD")) <> "NFOOD" THEN
                                 NVL(pd.ds_category,1)
         END)                                    as tot_categ_food
         ,COUNT(DISTINCT CASE WHEN UPPER(NVL(pd.cod_bus,"FOOD")) = "NFOOD" THEN
                                   NVL(pd.ds_category,1)
         END)                                    as tot_categ_nfood
         ,SUM(CASE WHEN UPPER(NVL(pd.cod_bus,"FOOD")) <> "NFOOD" THEN 
                        it.vl_item
                   END)                          AS vl_tot_categ_food
         ,SUM(CASE WHEN UPPER(NVL(pd.cod_bus,"FOOD")) = "NFOOD" THEN 
                        it.vl_item
                   END)                          AS vl_tot_categ_nfood
      from db_analytics_crm.tbl_fact_sale_item it
      left join
      db_analytics_crm.tbl_dim_product pd
        on it.safra = pd.safra
       and it.cod_product_rms = pd.cod_product_rms
      where it.safra = {var_safra}
      group by IT.num_document, it.safra) a
group by num_document, safra'''.format(var_safra=var_safra))

df_join_product.createOrReplaceTempView('tb_df_join_product')

print('#')
mostra_hora()

2020-08-13 09:13:50
# Processando safra 202008
2020-08-13 09:13:54


spark.sql('''select * from tb_df_join_product limit 20''').toPandas()

---
# Tabela_dim_sites

spark.sql('''
create table if not exists db_analytics_crm.tb_ind_item_site_temp5
(num_document         decimal(38,18)
,tot_gasto_hiper_m0   decimal(38,18)
,tot_gasto_market_m0  decimal(38,18)
,tot_gasto_bairro_m0  decimal(38,18)
,tot_gasto_express_m0 decimal(38,18)
,tot_gasto_ecom_m0    decimal(38,18)
,tot_gasto_posto_m0   decimal(38,18)
,tot_gasto_drogaria_m0 decimal(38,18)
) stored as orc
partitioned by (safra int)
''')

In [10]:
# processa safra específica
mostra_hora()
print('#')
print('# Processando safra ' + str(var_safra))

df_join_sites=spark.sql('''
select
   item.num_document
  ,{var_safra} as safra
  ,sum(CASE WHEN UPPER(loja.ds_site_format) = "LOJA HIPERMERCADO" OR 
                 UPPER(loja.ds_site_format) = "HIPERMERCADOS" THEN
                 item.vl_item
        END)                                             AS tot_gasto_hiper_m0
  ,sum(CASE WHEN UPPER(loja.ds_site_format) = 'SUPERMERCADOS' AND 
                 UPPER(loja.ds_site) LIKE '%MARKET%' AND 
                 loja.ini_site <> 'MVA' THEN 
                 item.vl_item
        END)                                             AS tot_gasto_market_m0
  ,sum(CASE WHEN (UPPER(loja.ds_site_format) = 'SUPERMERCADOS' AND 
                  UPPER(loja.ds_site) NOT LIKE '%MARKET%') 
                  OR
                  UPPER(loja.ini_site) = 'MVA' THEN
                  item.vl_item
                  END)                                        as tot_gasto_bairro_m0
  ,sum(CASE WHEN UPPER(loja.ds_site_format) = "EXPRESS" THEN
                 item.vl_item  
        END)                                             AS tot_gasto_express_m0
  ,sum(CASE WHEN UPPER(loja.ds_site_format) = "E-COMMERCE" THEN
                 item.vl_item
        END)                                             AS tot_gasto_ecom_m0
  ,sum(CASE WHEN UPPER(loja.ds_site_format) = "POSTOS" THEN
                 item.vl_item
        END)                                             AS tot_gasto_posto_m0
  ,sum(CASE WHEN UPPER(loja.ds_site_format) = "DROGARIA" THEN
                 item.vl_item
        END)                                             AS tot_gasto_drogaria_m0      
from  db_analytics_crm.tbl_fact_sale_item item
left join
db_analytics_crm.tbl_dim_site LOJA
 on loja.safra = item.safra
and item.cod_site = loja.cod_site
where item.safra = {var_safra}
group by item.num_document, item.safra
'''.format(var_safra=var_safra))

df_join_sites.createOrReplaceTempView('tb_itens_site_temp')

print('# Fim')
print('#')
mostra_hora()

2020-08-13 09:13:54
# Processando safra 202008
# Fim
2020-08-13 09:13:56


spark.sql('''select * from tb_itens_site_temp limit 20''').toPandas()

----
# tabela_final

mostra_hora()
spark.sql('''
create TABLE if not exists db_analytics_crm.tbl_indicadores_fact_client
(num_document               bigint
,gasto_medio_m0             decimal(38,18)
,ticket_medio_m0            decimal(38,18)
,tot_gasto_alimentar        decimal(38,18)
,tot_gasto_n_alimentar      decimal(38,18)
,qtd_categorias_compradas   bigint 
,qtd_categorias_alimentar   bigint 
,qtd_catgorias_n_alimentar  bigint
,tot_gasto_hiper            decimal(38,18)
,tot_gasto_market           decimal(38,18)
,tot_gasto_bairro           decimal(38,18)
,tot_gasto_express          decimal(38,18)
,tot_gasto_ecom             decimal(38,18)
,tot_gasto_posto            decimal(38,18)
,tot_gasto_drogaria         decimal(38,18) 
,tot_compras_m01            decimal(38,18)
,qtd_compras_m01            bigint
,tot_compras_m03            decimal(38,18)
,qtd_compras_m03            bigint
,tot_compras_m06            decimal(38,18)
,qtd_compras_m06            bigint
,tot_compras_m12            decimal(38,18)
,qtd_compras_m12            bigint
,dt_geracao_dado            date
) PARTITIONED BY ( safra int)
STORED AS ORC
TBLPROPERTIES (
"orc.compress"="SNAPPY",
"orc.compress.size"="262144",
"orc.create.index"="true",
"orc.row.index.stride"="10000",
"orc.stripe.size"="671088640",
"orc.bloom.filter.fpp"="0.01",
"orc.bloom.filter.columns"="num_document"
)
''')
mostra_hora()

In [11]:
mostra_hora()

print('# Gravando safra ' + str(var_safra))

spark.sql('''
insert overwrite table db_analytics_crm.tbl_indicadores_fact_client partition (safra)
select 
   cli.num_document
  ,cli.vl_ticket_acum_m0                          as gasto_medio_m0
  ,cli.vl_ticket_acum_m0 / cli.qty_ticket_acum_m0 as ticket_medio_m0
  ,prd.vl_tot_categ_food                          as tot_gasto_alimentar 
  ,prd.vl_tot_categ_nfood                         as tot_gasto_n_alimentar
  ,(prd.tot_categ_food + prd.tot_categ_nfood)     as qtd_categorias_compradas
  ,prd.tot_categ_food                             as qtd_categorias_alimentar
  ,prd.tot_categ_nfood                            as qtd_catgorias_n_alimentar 
  ,ste.tot_gasto_hiper_m0                         as tot_gasto_hiper  
  ,ste.tot_gasto_market_m0                        as tot_gasto_market
  ,ste.tot_gasto_bairro_m0                        as tot_gasto_bairro
  ,ste.tot_gasto_express_m0                       as tot_gasto_express
  ,ste.tot_gasto_ecom_m0                          as tot_gasto_ecom
  ,ste.tot_gasto_posto_m0                         as tot_gasto_posto
  ,ste.tot_gasto_drogaria_m0                      as tot_gasto_drogaria
  ,NVL(cli.vl_ticket_acum_m1  ,0)                 as tot_compras_m01  
  ,NVL(cli.qty_ticket_acum_m1 ,0)                 as qtd_compras_m01  
  ,NVL(cli.vl_ticket_acum_m3  ,0)                 as tot_compras_m03  
  ,NVL(cli.qty_ticket_acum_m3 ,0)                 as qtd_compras_m03 
  ,NVL(cli.vl_ticket_acum_m6  ,0)                 as tot_compras_m06  
  ,NVL(cli.qty_ticket_acum_m6 ,0)                 as qtd_compras_m06 
  ,NVL(cli.vl_ticket_acum_m12 ,0)                 as tot_compras_m12 
  ,NVL(cli.qty_ticket_acum_m12,0)                 as qtd_compras_m12
  ,current_date                                   as dt_geracao_dado
  ,cli.safra
from tb_df_fact_client_acum cli
left join
tb_df_join_product prd
 on cli.num_document = prd.num_document
and cli.safra = prd.safra
left join
tb_itens_site_temp ste
 on cli.num_document = ste.num_document
and cli.safra = ste.safra
where cli.safra = {var_safra}
'''.format(var_safra=var_safra))
print('# Fim')
mostra_hora()

2020-08-13 09:13:56
# Gravando safra 202008
# Fim
2020-08-13 09:23:25


mostra_hora()
#202001|27070646
spark.sql('''select safra, count(*) as total from db_analytics_crm.tbl_indicadores_fact_client
group by safra
order by safra 
''').show(truncate=False)
mostra_hora()

In [12]:
# 202004|1858584433.60000000000000000                           |
# 202008|274008581.08000000000000000 
# 202008|357915156.66000000000000000 
# 202008|370752372.75000000000000000 
# 202008|580928653.85000000000000000
mostra_hora()
spark.sql('''
select safra,(sum(tot_gasto_alimentar) + sum(tot_gasto_n_alimentar))
from db_analytics_crm.tbl_indicadores_fact_client
where safra between 201901 and 202008
group by safra
order by safra
'''.format(var_safra=var_safra)).show(truncate=False)
mostra_hora()

2020-08-13 09:23:25
+------+-------------------------------------------------------+
|safra |(sum(tot_gasto_alimentar) + sum(tot_gasto_n_alimentar))|
+------+-------------------------------------------------------+
|201901|1510125309.88000000000000000                           |
|201902|1483888564.74000000000000000                           |
|201903|1703212880.98000000000000000                           |
|201904|1711200388.07000000000000000                           |
|201905|1660992401.14000000000000000                           |
|201906|1580598194.93000000000000000                           |
|201907|1572398592.13000000000000000                           |
|201908|1687727046.80000000000000000                           |
|201909|1562191615.26000000000000000                           |
|201910|1648707471.70000000000000000                           |
|201911|1933794962.77000000000000000                           |
|201912|2263517665.87000000000000000                           |
|2020